In [45]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
from urlextract import URLExtract
import json
import sqlite3
import datetime
from tqdm import tqdm

In [2]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

In [3]:
result=requests.get('https://www.luluandsky.com/women.html?p=1',headers=headers)
get=result.content
soup=BeautifulSoup(get,'lxml')

## Womens Section

In [92]:
links=[
    ['https://www.luluandsky.com/women/dresses.html?p=','dresses'],
    ['https://www.luluandsky.com/women/tops.html?p=','tops'],
    ['https://www.luluandsky.com/women/lingerie.html?p=','lingerie'],
    ['https://www.luluandsky.com/women/coats-jackets.html?p=','coats-jackets'],
    ['https://www.luluandsky.com/women/cardigans-pullovers.html?p=','cardigans-pullovers'],
    ['https://www.luluandsky.com/women/skirts.html?p=','skirts'],
    ['https://www.luluandsky.com/women/sleepwear-innerwear.html?p=','sleepwear-innerwear'],
    ['https://www.luluandsky.com/women/beachwear.html?p=','beachwear'],
    ['https://www.luluandsky.com/women/jumpsuits-playsuits.html?p=','jumpsuits-playsuits'],
    ['https://www.luluandsky.com/women/jeans-trousers.html?p=','jeans-trousers'],
    ['https://www.luluandsky.com/women/shorts.html?p=','shorts'],
    ['https://www.luluandsky.com/women/denim.html?p=','denim'],
    ['https://www.luluandsky.com/women/two-piece-set.html?p=','two-piece-set'],
    ['https://www.luluandsky.com/bags.html?p=','bags'],
    ['https://www.luluandsky.com/shoes/trainers.html?p=','trainers'],
    ['https://www.luluandsky.com/shoes/boots.html?p=','boots'],
    ['https://www.luluandsky.com/shoes/casual-shoes.html?p=','casual-shoes'],
    ['https://www.luluandsky.com/shoes/flat-sandals.html?p=','flat-sandals'],
    ['https://www.luluandsky.com/shoes/flat-shoes.html?p=','flat-shoes'],
    ['https://www.luluandsky.com/shoes/heeled-sandals.html?p=','heeled-sandals'],
    ['https://www.luluandsky.com/shoes/heeled-shoes.html?p=','heeled-shoes'],
    ['https://www.luluandsky.com/shoes/pumps-peep-toes.html?p=','pumps-peep-toes'],
    ['https://www.luluandsky.com/shoes/wedges.html?p=','wedges'],
    ['https://www.luluandsky.com/accessories/sunglasses.html?p=','sunglasses'],
    ['https://www.luluandsky.com/accessories/belts.html?p=','belts'],
    ['https://www.luluandsky.com/accessories/hats-caps.html?p=','hats-caps'],
    ['https://www.luluandsky.com/accessories/jewellery.html?p=','jewellery'],
    ['https://www.luluandsky.com/accessories/gloves-and-earmuffs.html?p=','gloves-and-earmuffs'],
    ['https://www.luluandsky.com/accessories/socks-and-stockings.html?p=','socks-and-stockings']
]

In [93]:
prod_links=[]
for i in links:
    try:
        result=requests.get(i[0]+str(1),headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        c=math.ceil(int(soup.find('div',{'class':'categorycount'}).text.split()[0])/24)
        for j in tqdm(range(1,c+1)):
            try:
                result=requests.get(i[0]+str(j),headers=headers)
                get=result.content
                soup=BeautifulSoup(get,'lxml')
                prod=soup.find_all('strong',{'class':'product name product-item-name'})
                for k in prod:
                    try:
                        li=[]
                        li.append(k.find('a').attrs['href'])
                        li.append(i[1])
                        prod_links.append(li)
                    except:
                        pass
            except:
                pass
    except:
        pass
    

100%|██████████| 1/1 [00:00<00:00,  2.00it/s]


In [177]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [178]:
err=[]
for i in tqdm(prod_links):
    try:
        result = requests.get(i[0],headers=headers)
        get = result.content
        soup = BeautifulSoup(get,"lxml")
        product_name=soup.find('div',{'class':'product-name'}).find('h2').text
        product_brand=soup.find('div',{'class':'model-details'}).find_all('div',{'class':'std'})[1].text.split(':')[1].lstrip().rstrip()
        product_link=i[0]
        product_cat=i[1]
        product_des=soup.find('ul',{'class':'product-attributes'}).find('div').text
        product_price=int(''.join(price.find('div',{'class':'price'}).text.split('₹')[1].split(',')))
        product_mrp=int(''.join(price.find('div',{'class':'old-price'}).text.split('₹')[1].split(',')))
        li=soup.find_all('a',{'class':'photoswipe'})
        sizes=soup.find('ul',{'class':'configurable-swatch-list size-list'}).find_all('li')
        size=[]
        for k in sizes:
            size.append(k.text)   
        size_avl=','.join(size)
        img=soup.find('div',{'class':'product media'})
        image_split=img.find('img').attrs['src'].replace('305x453','707x1000').split('_')
        if(len(image_split)>1):
            images=[]
            for l in range(1,7):
                images.append(image_split[0]+'_'+str(l)+'_'+image_split[2])
        else:
            image_split=img.find('img').attrs['src'].replace('305x453','707x1000').split('-1.jpg')[0]
            images=[]
            for l in range(1,7):
                images.append(image_split[0]+'-'+str(l)+'.jpg')
    
    except:
        err.append(i)
        pass
    
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Women')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)
        
    

100%|██████████| 2040/2040 [1:16:08<00:00,  2.24s/it]


In [167]:
for i in tqdm(prod_links):
    result = requests.get(i[0],headers=headers)
    get = result.content
    soup = BeautifulSoup(get,"lxml")
    product_name=soup.find('div',{'class':'product-name'}).find('h2').text
    product_brand=soup.find('div',{'class':'model-details'}).find_all('div',{'class':'std'})[1].text.split(':')[1].lstrip().rstrip()
    product_link=i[0]
    product_cat=i[1]
    product_des=soup.find('ul',{'class':'product-attributes'}).find('div').text
    product_price=int(''.join(price.find('div',{'class':'price'}).text.split('₹')[1].split(',')))
    product_mrp=int(''.join(price.find('div',{'class':'old-price'}).text.split('₹')[1].split(',')))
    li=soup.find_all('a',{'class':'photoswipe'})
    sizes=soup.find('ul',{'class':'configurable-swatch-list size-list'}).find_all('li')
    size=[]
    for k in sizes:
        size.append(k.text)   
    size_avl=','.join(size)
    img=soup.find('div',{'class':'product media'})
    image_split=img.find('img').attrs['src'].replace('305x453','707x1000').split('_')
    if(len(image_split)>1):
        images=[]
        for l in range(1,7):
            images.append(image_split[0]+'_'+str(l)+'_'+image_split[2])
    else:
        image_split=img.find('img').attrs['src'].replace('305x453','707x1000').split('-1.jpg')[0]
        images=[]
        for l in range(1,7):
            images.append(image_split[0]+'-'+str(l)+'.jpg')
        
    primary_link=images[0]
    secondary_link=','.join(images[1:])

  2%|▏         | 33/2040 [01:12<1:13:10,  2.19s/it]


IndexError: list index out of range

In [142]:
out=pd.DataFrame({
    
    "Website":'https://www.luluandsky.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [180]:
con=sqlite3.connect("lulu&sky.db")  
cur=con.cursor()
cur.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
for i in range(len(Product_Link)):
    try:
        cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.luluandsky.com/',Product_Link[i],Product_Name[i],Product_Brand[i],Cat_list[i],Size_Available[i],Price[i],MRP[i],Gender[i],Description[i],Primary_Image_Link[i],Secondary_Image_Link[i],''))
    except:
        pass
con.commit()
con.close()